In [1]:
import numpy as np
import pandas as pd
import sklearn

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [2]:
# step1：去掉数据不完整的行
df = pd.read_csv("2017.csv").dropna()
# step2：性别、是否吸烟等特征转one-hot
df = pd.get_dummies(df, columns=['sex','SMK_STAT_TYPE_RSPS_CD'])
# step3: 去掉creatinine高于2.5的样本
df = df.loc[(df["CREATININE "]<=2.5)&(df["CREATININE "]>=0.0)]
# step4: 切分训练集与测试集
df_train = df[0:900000]; df_test = df[900000:990112] 

In [3]:
# step5: 训练集与测试集下采样
intervals = [(i * 0.1 + 0.05, i * 0.1 + 0.15) for i in range(25)]

temp_train, temp_test = [], []

for index, (low, high) in enumerate(intervals):
    
    df_sample_train = df_train.loc[(df_train["CREATININE "] > low) & (df_train["CREATININE "] < high)]
    df_sample_test  = df_test.loc[(df_test["CREATININE "] > low) & (df_test["CREATININE "] < high)]
    
    if index in range(3, 14): df_sample_train = df_sample_train.sample(2500)
    if index in range(3, 16): df_sample_test  = df_sample_test.sample(100)
    
    temp_train.append(df_sample_train)
    temp_test.append(df_sample_test)

df_train = pd.concat(temp_train, ignore_index=True)
df_test = pd.concat(temp_test, ignore_index=True)

# step6: 选取特征x与目标y
y_train = df_train["CREATININE "].values.reshape(-1, 1)
x_train = df_train.drop(columns=["id", "sido", "CREATININE ", "GFR", "stage", "height", "weight"]).values

y_test = df_test["CREATININE "].values.reshape(-1, 1)
x_test = df_test.drop(columns=["id", "sido", "CREATININE ", "GFR", "stage", "height", "weight"]).values

In [7]:
# step7: 训练RandomForest模型，计算预测值与真实值的R2
model = RandomForestRegressor(
    criterion="squared_error",n_estimators=100,max_depth=10, 
    min_samples_split=40,min_samples_leaf=7,
    max_features=7,oob_score=True, random_state=10
)
model.fit(x_train, y_train)

print("R2指数：",model.score(x_test, y_test))

/Users/wangweilun/miniconda3/envs/ChronicKidneyDisease/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


R2指数： 0.5360178356736531
